# Cargamos librerias y conectamos la base de datos

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import time, json, random
from random import randrange
from datetime import timedelta, datetime
from random import randint
from lorem_text import lorem
from flask import jsonify

es = Elasticsearch("http://127.0.0.1:9200")

print(f"Connected to ElasticSearch cluster `{es.info().body['cluster_name']}`")

Connected to ElasticSearch cluster `docker-cluster`


# Generación de datos aleatorios

In [2]:
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [3]:

d1 = datetime.strptime('1/1/1950 12:59 PM', '%m/%d/%Y %I:%M %p')
d2 = datetime.strptime('1/1/2022 01:00 AM', '%m/%d/%Y %I:%M %p')
print(random_date(d1, d2))  

2004-03-18 04:22:39


In [4]:
descripciones = [
    'Transferencia interbancaria SIMPLE',
    'DEB.CTA.P/C.INTERNET',
    'Debito Cta por ACH',
    'Retiro en efectivo',
    'AB.CTA.P/MOVIL',
    'Pago en interés',
    'Debito Cta por compra',
    'Debito Cta por pago de Seguro',
    'Retención IVA'
]

# Creación de índices

In [10]:
mapping = {
    "mappings": {
        "properties": {
            "fecha": {
                "type": "date"
            },
            "agencia": {
                "type": "keyword"
            },
            "monto": {
                "type": "float"
            },
            "descripcion": {
                "type": "text"
            },
            "saldo": {
                "type": "float"
            },
            "nota": {
                "type": "text"
            },
        }
    }
}

response = es.indices.create(
    index="extracto_cuenta",
    mappings = mapping,
    ignore=400 # ignore 400 already exists code
)

/tmp/ipykernel_18717/5996740.py:26: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = es.indices.create(


# Aproximadamente 100 000 registros

## Carga de datos

### De forma individual

In [11]:
start_time = time.time()

for i in range(50000):
    fecha = random_date(d1, d2)
    agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
    monto = round(random.uniform(10.0, 4000.0), 2)
    descripcion = random.choice(descripciones)
    saldo = round(random.uniform(0.0, 10000000.0), 2)
    nota = lorem.words(randint(1,13))

    insert_data = {
        'fecha': fecha,
        'agencia': agencia,
        'monto': monto,
        'descripcion': descripcion,
        'saldo': saldo,
        'nota': nota
    }
    
    es.index(index="extracto_cuenta", document=insert_data)

end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 1:39:44.988039


### De forma masiva

In [5]:
def gendata():
    for i in range(500000):
        fecha = random_date(d1, d2)
        agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
        monto = round(random.uniform(10.0, 4000.0), 2)
        descripcion = random.choice(descripciones)
        saldo = round(random.uniform(0.0, 10000000.0), 2)
        nota = lorem.words(randint(1,13))

        yield {
            "_index": "extracto_cuenta",
            'fecha': fecha,
            'agencia': agencia,
            'monto': monto,
            'descripcion': descripcion,
            'saldo': saldo,
            'nota': nota
        }

start_time = time.time()
helpers.bulk(es, gendata())
end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 0:00:24.350864


### Un solo dato

In [6]:
start_time = time.time()

fecha = random_date(d1, d2)
agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
monto = round(random.uniform(10.0, 4000.0), 2)
descripcion = random.choice(descripciones)
saldo = round(random.uniform(0.0, 10000000.0), 2)
nota = lorem.words(randint(1,13))

insert_data = {
    'fecha': fecha,
    'agencia': agencia,
    'monto': monto,
    'descripcion': descripcion,
    'saldo': saldo,
    'nota': nota
}

es.index(index="extracto_cuenta", document=insert_data)

end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 0:00:00.124868


In [7]:
!curl -XGET http://localhost:9200/extracto_cuenta/_count

{"count":100001,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}

## Consultas a la base de datos

### Búsqueda por fecha y agencia

In [12]:
def busqueda_fecha_agencia(month, year, agencia):
    from_date_str = f"{year}-{month}-01" + "T00:00:00+00:00"
    to_date_str = f"{year}-{month}-30" + "T23:59:59+00:00"
    query = {
            "bool": {
                "must": [
                    { "match": { "agencia": f"{agencia}" } },
                    {"range" : {
                        "fecha": { "gte" : f"{from_date_str}", "lte" : f"{to_date_str}"}}
                    }
                ]
            }
    }

    return es.search(index='extracto_cuenta', query=query)

In [13]:
start_time = time.time()
results = busqueda_fecha_agencia(month="05", year = "2021", agencia = "Santa Cruz")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])

Se encontraron 9 en un tiempo de 0:00:00.012294
{'fecha': '2021-05-23T11:52:00', 'agencia': 'Santa Cruz', 'monto': 974.58, 'descripcion': 'Pago en interés', 'saldo': 515906.12, 'nota': 'consectetur quae facilis fugit earum rerum voluptates similique excepturi in ipsum asperiores'}
{'fecha': '2021-05-09T01:21:36', 'agencia': 'Santa Cruz', 'monto': 3954.92, 'descripcion': 'Retención IVA', 'saldo': 1562951.71, 'nota': 'molestiae assumenda'}
{'fecha': '2021-05-09T11:45:46', 'agencia': 'Santa Cruz', 'monto': 2992.84, 'descripcion': 'Pago en interés', 'saldo': 1502314.46, 'nota': 'quo eligendi'}
{'fecha': '2021-05-28T02:08:48', 'agencia': 'Santa Cruz', 'monto': 1659.51, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 2533581.24, 'nota': 'ab expedita quos totam sunt sed neque et illum at cumque'}
{'fecha': '2021-05-28T21:16:01', 'agencia': 'Santa Cruz', 'monto': 2675.89, 'descripcion': 'Debito Cta por pago de Seguro', 'saldo': 2842735.11, 'nota': 'vel optio voluptates voluptate ratione cum repudian

### Búsqueda por montos mayores a determinado número y agencia

In [14]:
def busqueda_monto_mayor(monto, agencia):
    query = {
            "bool": {
            "must": [
                { "match": 
                    { "agencia": f"{agencia}" } 
                },
                {"range" : {
                "monto": { "gte" : f"{monto}"}}
                }
            ]
            }
    }

    return es.search(index='extracto_cuenta', query=query)

In [15]:
start_time = time.time()
results = busqueda_monto_mayor(monto=3000, agencia = "Cochabamba")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])

Se encontraron 3030 en un tiempo de 0:00:00.016266
{'fecha': '1997-12-11T18:08:54', 'agencia': 'Cochabamba', 'monto': 3715.81, 'descripcion': 'Debito Cta por ACH', 'saldo': 4213381.94, 'nota': 'voluptatem dolorem aliquam sint consectetur iste mollitia doloribus soluta ducimus inventore'}
{'fecha': '2021-06-16T14:17:11', 'agencia': 'Cochabamba', 'monto': 3779.35, 'descripcion': 'Debito Cta por compra', 'saldo': 3797420.61, 'nota': 'ratione voluptate excepturi deserunt eveniet soluta consequatur aut aliquam vero laudantium atque'}
{'fecha': '1964-12-21T11:30:15', 'agencia': 'Cochabamba', 'monto': 3573.06, 'descripcion': 'Transferencia interbancaria SIMPLE', 'saldo': 9010423.36, 'nota': 'earum sint laboriosam tempore sed saepe'}
{'fecha': '2021-05-14T17:47:23', 'agencia': 'Cochabamba', 'monto': 3996.89, 'descripcion': 'Retiro en efectivo', 'saldo': 7522942.78, 'nota': 'id porro nobis quasi'}
{'fecha': '2001-09-26T04:04:08', 'agencia': 'Cochabamba', 'monto': 3138.3, 'descripcion': 'Debito 

### Búsqueda considerando varios campos

In [84]:
def busqueda_compleja(saldo_min, saldo_max, agencia1, agencia2, descripcion):
    query = {
            "bool": {
                "must": [
                    {
                        "bool": {
                            "should": [
                                {"match": {"agencia": f"{agencia1}"}},
                                {"match": {"agencia": f"{agencia2}"}}
                            ]
                        }
                    },
                    {
                        "bool": {
                            "must": [
                                {
                                    "range" : {
                                        "saldo": { "gte" : f"{saldo_min}", "lte": f"{saldo_max}"}
                                    }
                                }
                            ]
                        }
                    },
                    {
                        "match": {"descripcion": f"{descripcion}"}
                    }
                ]
            }
    }

    return es.search(index='extracto_cuenta', query=query)

In [40]:
start_time = time.time()
results = busqueda_compleja(saldo_min=1000, saldo_max=1500000, agencia1="La Paz", agencia2="Pando", descripcion="Retiro en efectivo")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])

Se encontraron 4174 en un tiempo de 0:00:00.019802
{'fecha': '1979-08-03T09:58:38', 'agencia': 'Pando', 'monto': 3443.92, 'descripcion': 'Retiro en efectivo', 'saldo': 7149020.16, 'nota': 'iusto rem ipsam maxime quaerat'}
{'fecha': '1994-06-22T22:52:03', 'agencia': 'Pando', 'monto': 2414.66, 'descripcion': 'Retiro en efectivo', 'saldo': 6308997.27, 'nota': 'repellat excepturi nulla dignissimos'}
{'fecha': '1977-02-14T06:10:16', 'agencia': 'Pando', 'monto': 3739.83, 'descripcion': 'Retiro en efectivo', 'saldo': 5575542.39, 'nota': 'debitis facere culpa vero repellat qui'}
{'fecha': '1983-02-11T19:04:14', 'agencia': 'Pando', 'monto': 2089.46, 'descripcion': 'Retiro en efectivo', 'saldo': 8475804.66, 'nota': 'perspiciatis alias quos voluptatem natus'}
{'fecha': '1952-09-29T21:32:45', 'agencia': 'Pando', 'monto': 1486.37, 'descripcion': 'Retiro en efectivo', 'saldo': 7364448.85, 'nota': 'minima'}
{'fecha': '1966-07-22T19:03:02', 'agencia': 'Pando', 'monto': 2425.09, 'descripcion': 'Retiro 

### Agregaciones: Monto promedio por agencia

In [64]:
def promedio_monto_agencia(agencia):
    aggs = {
            "aggs" : {
                    "avg" : {
                        "field" : "monto",
                    } 
            }
        }

    query = {
        "match": {"agencia": "La Paz"}
    }



    return es.search(index='extracto_cuenta', query=query, aggs=aggs)

In [65]:
start_time = time.time()
results = promedio_monto_agencia(agencia="La Paz")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

print(results['hits'])
print(results['aggregations'])

Se encontraron 10000 en un tiempo de 0:00:00.007124
{'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 2.0789416, 'hits': [{'_index': 'extracto_cuenta', '_id': '9xTjM4QBzIg4_3FeroEN', '_score': 2.0789416, '_source': {'fecha': '2011-07-13T02:34:40', 'agencia': 'La Paz', 'monto': 2794.73, 'descripcion': 'Retención IVA', 'saldo': 5980936.73, 'nota': 'repellendus id quibusdam eaque sunt dolore modi doloremque in porro'}}, {'_index': 'extracto_cuenta', '_id': '_BTjM4QBzIg4_3FesIFD', '_score': 2.0789416, '_source': {'fecha': '1990-05-28T15:42:37', 'agencia': 'La Paz', 'monto': 2669.0, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 1837707.74, 'nota': 'consequuntur ab est asperiores ea eveniet consectetur aliquam nemo assumenda'}}, {'_index': 'extracto_cuenta', '_id': 'JRTjM4QBzIg4_3FewIKN', '_score': 2.0789416, '_source': {'fecha': '1983-08-22T10:38:09', 'agencia': 'La Paz', 'monto': 2525.02, 'descripcion': 'Retiro en efectivo', 'saldo': 9484348.41, 'nota': 'natus'}}, {'_index': 'extract

## Actualización de datos

### Actualización de montos en un rango determinado de fechas

In [66]:
def actualizacion_fecha_monto(min_date, max_date, monto):
      from_date_str = f"{min_date}" + "T00:00:00+00:00"
      to_date_str = f"{max_date}" + "T23:59:59+00:00"
   

      script = {
            "source": f"ctx._source['monto'] = {monto}; ctx._source['nota'] = 'Actualización de monto por fecha'; ",
            "lang": "painless"
      }

      query = {
                  "range" : {
                  "fecha": { "gte" : f"{min_date}", "lte" : f"{max_date}" },
                  }
      }

      return es.update_by_query(query=query, script=script, index='extracto_cuenta')

In [67]:
start_time = time.time()
results = actualizacion_fecha_monto(min_date = "2022-01-01", max_date = "2022-05-30", monto = 7000)
end_time = time.time()


print(f"Se actualizaron {results['total']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

Se actualizaron 1 en un tiempo de 0:00:00.534087


In [68]:
query = {
    "range" : {
                  "fecha": { "gte" : f"2022-01-01", "lte" : f"2022-05-30" },
      }
  }

results = es.search(index='extracto_cuenta', query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Retención IVA', 'fecha': '2022-01-01T00:47:54', 'monto': 7000, 'saldo': 4493676.34, 'nota': 'Actualización de monto por fecha', 'agencia': 'Tarija'}
Se encontraron: 1 resultados


## Eliminar datos

### Eliminar saldos menores a determinado número

In [69]:
def eliminar_saldo_min(saldo_min):

    query = {
        "range" : {
                  "saldo": { "lte" : f"{saldo_min}"},
      }
    }

    return es.delete_by_query(index = 'extracto_cuenta', query = query)

In [70]:
start_time = time.time()
results = eliminar_saldo_min(saldo_min= 100)
end_time = time.time()

print(results)
print(f"Se eliminaron {results['total']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

{'took': 326, 'timed_out': False, 'total': 2, 'deleted': 2, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}
Se eliminaron 2 en un tiempo de 0:00:00.334268


In [71]:
query = {
    "range" : {
                  "saldo": { "lte" : 100},
      }
  }

results = es.search(index='extracto_cuenta', query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

Se encontraron: 0 resultados


# Mas de 10 millon de registros

## Carga de datos

In [72]:
def gendata():
    for i in range(500000):
        fecha = random_date(d1, d2)
        agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
        monto = round(random.uniform(10.0, 4000.0), 2)
        descripcion = random.choice(descripciones)
        saldo = round(random.uniform(0.0, 10000000.0), 2)
        nota = lorem.words(randint(1,13))

        yield {
            "_index": "extracto_cuenta",
            'fecha': fecha,
            'agencia': agencia,
            'monto': monto,
            'descripcion': descripcion,
            'saldo': saldo,
            'nota': nota
        }

start_time = time.time()

for _ in range(0,25):
    try:
        helpers.bulk(es, gendata())
    except: 
        pass
    
end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 1:12:24.442911


In [74]:
def gendata():
    for i in range(500000):
        fecha = random_date(d1, d2)
        agencia = random.choice(['La Paz', 'Oruro', 'Cochabamba', 'Chuquisaca', 'Pando', 'Beni', 'Santa Cruz', 'Tarija'])
        monto = round(random.uniform(10.0, 4000.0), 2)
        descripcion = random.choice(descripciones)
        saldo = round(random.uniform(0.0, 10000000.0), 2)
        nota = lorem.words(randint(1,13))

        yield {
            "_index": "extracto_cuenta",
            'fecha': fecha,
            'agencia': agencia,
            'monto': monto,
            'descripcion': descripcion,
            'saldo': saldo,
            'nota': nota
        }

start_time = time.time()

for _ in range(0,2):
    try:
        helpers.bulk(es, gendata())
    except: 
        pass
    
end_time = time.time()

# Mostramos los resultados en consola
print(f'Tiempo de carga de datos: {str(timedelta(seconds = (end_time - start_time)))}')

Tiempo de carga de datos: 0:05:06.937152


In [75]:
!curl -XGET http://localhost:9200/extracto_cuenta/_count

{"count":10469500,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}

## Consulta de datos

### Búsqueda por fecha y agencia

In [76]:
start_time = time.time()
results = busqueda_fecha_agencia(month="05", year = "2021", agencia = "Santa Cruz")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])

Se encontraron 1502 en un tiempo de 0:00:02.454354
{'fecha': '2021-05-07T03:24:29', 'agencia': 'Santa Cruz', 'monto': 3461.86, 'descripcion': 'Transferencia interbancaria SIMPLE', 'saldo': 9893162.25, 'nota': 'blanditiis itaque cupiditate explicabo'}
{'fecha': '2021-05-26T13:46:29', 'agencia': 'Santa Cruz', 'monto': 1247.1, 'descripcion': 'Transferencia interbancaria SIMPLE', 'saldo': 6314865.37, 'nota': 'officiis autem adipisci officia esse corrupti'}
{'fecha': '2021-05-27T07:17:19', 'agencia': 'Santa Cruz', 'monto': 3327.88, 'descripcion': 'Debito Cta por pago de Seguro', 'saldo': 5857848.61, 'nota': 'rem asperiores quisquam suscipit dicta voluptatem deleniti officia sit minus alias nisi'}
{'fecha': '2021-05-19T23:32:55', 'agencia': 'Santa Cruz', 'monto': 99.86, 'descripcion': 'Pago en interés', 'saldo': 8595914.55, 'nota': 'alias'}
{'fecha': '2021-05-19T12:24:33', 'agencia': 'Santa Cruz', 'monto': 3076.24, 'descripcion': 'Pago en interés', 'saldo': 3910177.71, 'nota': 'nam sapiente 

### Búsqueda por montos mayores a determinado número y agencia

In [77]:
start_time = time.time()
results = busqueda_monto_mayor(monto=3000, agencia = "Cochabamba")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])

Se encontraron 10000 en un tiempo de 0:00:02.061207
{'fecha': '1977-09-09T03:37:13', 'agencia': 'Cochabamba', 'monto': 3516.03, 'descripcion': 'Retiro en efectivo', 'saldo': 4780436.8, 'nota': 'dolorum aut molestiae libero adipisci modi corrupti'}
{'fecha': '2013-07-11T15:05:55', 'agencia': 'Cochabamba', 'monto': 3648.64, 'descripcion': 'Retiro en efectivo', 'saldo': 5900955.72, 'nota': 'corrupti blanditiis libero rerum sed perspiciatis nisi quisquam'}
{'fecha': '1972-12-03T17:56:35', 'agencia': 'Cochabamba', 'monto': 3413.83, 'descripcion': 'DEB.CTA.P/C.INTERNET', 'saldo': 6966391.74, 'nota': 'veniam minus sequi ullam non saepe est'}
{'fecha': '1961-12-01T15:42:51', 'agencia': 'Cochabamba', 'monto': 3424.54, 'descripcion': 'Debito Cta por compra', 'saldo': 3756687.42, 'nota': 'vitae consequatur'}
{'fecha': '2015-05-31T02:35:24', 'agencia': 'Cochabamba', 'monto': 3964.37, 'descripcion': 'Debito Cta por ACH', 'saldo': 5471718.23, 'nota': 'vero'}
{'fecha': '1980-02-27T01:22:43', 'agencia

### Búsqueda considerando varios campos

In [99]:
def busqueda_compleja(saldo_min, saldo_max, agencia1, agencia2, descripcion):
    query = {
            "bool": {
                "must": [
                    {
                        "bool": {
                            "should": [
                                {"match": {"agencia": f"{agencia1}"}},
                                {"match": {"agencia": f"{agencia2}"}}
                            ]
                        }
                    },
                    {
                        "bool": {
                            "must": [
                                {
                                    "range" : {
                                        "monto": { "gte" : f"{saldo_min}", "lte": f"{saldo_max}"}
                                    }
                                }
                            ]
                        }
                    },
                    {
                        "match": {"descripcion": f"{descripcion}"}
                    }
                ]
            }
    }

    return es.search(index='extracto_cuenta', query=query)

In [101]:
start_time = time.time()
results = busqueda_compleja(saldo_min=1000, saldo_max=1500000, agencia1="Beni", agencia2="Pando", descripcion="Retiro en efectivo")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

for hit in results['hits']['hits']:
    print(hit["_source"])

Se encontraron 10000 en un tiempo de 0:00:03.324822
{'fecha': '1992-03-27T22:37:39', 'agencia': 'Beni', 'monto': 3514.88, 'descripcion': 'Retiro en efectivo', 'saldo': 7564017.42, 'nota': 'maiores blanditiis impedit libero laborum nostrum suscipit nobis enim corrupti dicta eveniet sit'}
{'fecha': '1955-08-11T05:50:04', 'agencia': 'Beni', 'monto': 1515.24, 'descripcion': 'Retiro en efectivo', 'saldo': 6969180.17, 'nota': 'accusamus quidem consequuntur doloremque veniam eum eveniet'}
{'fecha': '1954-07-25T13:59:31', 'agencia': 'Beni', 'monto': 2324.39, 'descripcion': 'Retiro en efectivo', 'saldo': 5438280.1, 'nota': 'sunt voluptas maxime praesentium tempora ullam nostrum sit molestias similique'}
{'fecha': '1991-12-03T16:13:37', 'agencia': 'Beni', 'monto': 2657.4, 'descripcion': 'Retiro en efectivo', 'saldo': 8994435.01, 'nota': 'molestias deserunt dignissimos et'}
{'fecha': '2007-11-12T12:23:26', 'agencia': 'Beni', 'monto': 1046.99, 'descripcion': 'Retiro en efectivo', 'saldo': 9289430.

### Agregaciones: Monto promedio por agencia

In [102]:
start_time = time.time()
results = promedio_monto_agencia(agencia="La Paz")
end_time = time.time()

print(f"Se encontraron {results['hits']['total']['value']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

print(results['hits'])
print(results['aggregations'])

Se encontraron 10000 en un tiempo de 0:00:02.052796
{'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 3.3407502, 'hits': [{'_index': 'extracto_cuenta', '_id': 'RVyhNYQBKdMpO4lrUdjb', '_score': 3.3407502, '_source': {'fecha': '1965-10-14T12:51:17', 'agencia': 'La Paz', 'monto': 1055.86, 'descripcion': 'Retención IVA', 'saldo': 259714.1, 'nota': 'distinctio cupiditate sed impedit quasi est'}}, {'_index': 'extracto_cuenta', '_id': 'UVyhNYQBKdMpO4lrUdjb', '_score': 3.3407502, '_source': {'fecha': '1972-09-17T08:22:13', 'agencia': 'La Paz', 'monto': 3804.55, 'descripcion': 'AB.CTA.P/MOVIL', 'saldo': 5197980.07, 'nota': 'consectetur ipsa velit repudiandae ullam'}}, {'_index': 'extracto_cuenta', '_id': 'XVyhNYQBKdMpO4lrUdjb', '_score': 3.3407502, '_source': {'fecha': '1960-08-12T05:47:43', 'agencia': 'La Paz', 'monto': 1514.74, 'descripcion': 'Debito Cta por pago de Seguro', 'saldo': 6499602.55, 'nota': 'voluptates corrupti'}}, {'_index': 'extracto_cuenta', '_id': 'YVyhNYQBKdMpO4lrU

## Actualización de datos

### Actualización de montos en un rango determinado de fechas

In [103]:
start_time = time.time()
results = actualizacion_fecha_monto(min_date = "2022-01-01", max_date = "2022-05-30", monto = 7000)
end_time = time.time()


print(f"Se actualizaron {results['total']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

Se actualizaron 20 en un tiempo de 0:00:03.214966


In [104]:
query = {
    "range" : {
                  "fecha": { "gte" : f"2022-01-01", "lte" : f"2022-05-30" },
      }
  }

results = es.search(index='extracto_cuenta', query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Debito Cta por pago de Seguro', 'fecha': '2022-01-01T00:30:33', 'monto': 7000, 'saldo': 4499644.88, 'nota': 'Actualización de monto por fecha', 'agencia': 'Oruro'}
{'descripcion': 'DEB.CTA.P/C.INTERNET', 'fecha': '2022-01-01T00:11:42', 'monto': 7000, 'saldo': 5689893.75, 'nota': 'Actualización de monto por fecha', 'agencia': 'Beni'}
{'descripcion': 'Retención IVA', 'fecha': '2022-01-01T00:01:34', 'monto': 7000, 'saldo': 7351642.72, 'nota': 'Actualización de monto por fecha', 'agencia': 'Pando'}
{'descripcion': 'Retiro en efectivo', 'fecha': '2022-01-01T00:35:35', 'monto': 7000, 'saldo': 193253.51, 'nota': 'Actualización de monto por fecha', 'agencia': 'Santa Cruz'}
{'descripcion': 'Pago en interés', 'fecha': '2022-01-01T00:27:13', 'monto': 7000, 'saldo': 151798.37, 'nota': 'Actualización de monto por fecha', 'agencia': 'Cochabamba'}
{'descripcion': 'Retención IVA', 'fecha': '2022-01-01T00:49:50', 'monto': 7000, 'saldo': 5993279.21, 'nota': 'Actualización de monto por f

## Eliminar datos

### Eliminar saldos menores a determinado número

In [105]:
start_time = time.time()
results = eliminar_saldo_min(saldo_min= 100)
end_time = time.time()

print(results)
print(f"Se eliminaron {results['total']} en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")

{'took': 6080, 'timed_out': False, 'total': 116, 'deleted': 116, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}
Se eliminaron 116 en un tiempo de 0:00:06.084581


In [106]:
query = {
    "range" : {
                  "saldo": { "lte" : 100},
      }
  }

results = es.search(index='extracto_cuenta', query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

Se encontraron: 0 resultados
